In [7]:
import numpy as np

In [ ]:
# Task 1
# Find the roots of square equation by gradient descent
# x ** 2 - 6 * x + 4 = 0


In [ ]:
# возвести в квадрат
# посчитать производную
# надо начать движение от начальной точки в направлении антградиента с заданным шагом
# x = x - lr * grad(x)
# всегда ли сойдемся за приемлемое количество шагов?
# важна ли начальная точка?
# как найти второй корень?
# как вляет ЛР?

In [ ]:
# Task 2
# Realize forward and backward pass for linear layer with sigmoid activation

In [54]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

def sigmoid_backward(da, x):
    sig = sigmoid(x)
    
    return da * sig * (1 - sig)

def relu(x):
    return np.maximum(0., x)

def relu_backward(da, x):
    da = np.array(da, copy = True)
    da[x <= 0] = 0;
    return da;

In [55]:
def mse_loss(t, y):
    return (t - y) ** 2

def d_mse_loss(t, y):
    return 2 * (y - t) 


In [56]:
class LinearLayer:
    def __init__(self, n_inp, n_out, activation='sigmoid'):
        self.w = np.random.randn(n_out, n_inp) * 0.1
        self.b = np.random.randn(n_out, 1) * 0.1
        if activation == 'sigmoid':
            self.activ = sigmoid
        if activation == 'relu':
            self.activ = relu
        elif activation == 'None':
            self.activ = None
        else:
            raise Exception(f'Unknown activation "{activation}"')
        self._clear_state()

    def _clear_state(self):
        self.lin = None
        self.inp = None
        self.d_w = None
        self.d_b = None

    def forward(self, x):
        self.inp = x
        self.lin = np.dot(self.w, x) + self.b
        activ = self.activ(self.lin) if self.activ is not None else self.lin

        return activ

    def backward(self, grad): # grad = d L / d z    Dout 
        # grad * dz / d lin
        if self.activ == sigmoid:
            grad_lin = sigmoid_backward(grad, self.lin) 
        elif self.activ == relu:
            grad_lin = relu_backward(grad, self.lin)
        else:
            grad_lin = grad
        # grad_lin * d lin / d w 
        m = self.inp.shape[1]
        self.d_w = np.dot(grad_lin, self.inp.T) / m    # d_in dOut
        # grad_lin * d lin / d b 
        self.d_b = np.sum(grad_lin, axis=1, keepdims=True) / m

        grad = np.dot(self.w.T, grad_lin)

        return grad

# pred = model(x)
# loss = criterion(pred, target)
# grad = d loss / d pred
# model.backward(grad)

In [48]:
from typing import Tuple

class Model:
    def __init__(self, arch: Tuple[Tuple[int, int]], activation):
        self.layers = []
        for i, p in enumerate(arch):
            self.layers.append(
                LinearLayer(p[0], p[1], 
                            activation=activation if i < len(arch)-1 else 'None')
                )
        self._clear_state()
    
    def _clear_state(self):
        for l in self.layers:
            l._clear_state()

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        
        return x

    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

        return grad 

In [ ]:
# Task 3
# Realize SGD Momentum optimizer
# velocity = momentum * velocity - lr * gradient
# w = w + velocity

In [ ]:
#для одного слоя
class SGDMomentum:
    def __init__(self, model: LinearLayer, lr=0.001, momentum=0.99):
        self.lr = lr
        self.m = momentum
        self.model = model

        self.vel_w = np.zeros_like(model.w)
        self.vel_b = np.zeros_like(model.b)

    def step(self):
        self.vel_w = self.m * self.vel_w - self.lr * self.model.d_w
        self.vel_b = self.m * self.vel_b - self.lr * self.model.d_b

        self.model.w += self.vel_w
        self.model.b += self.vel_b

        




In [60]:
#для всей модели
class SGDMomentum:
    def __init__(self, model: Model, lr= 0.0001, momentum=0.0):
        self.model = model
        self.lr = lr
        self.m = momentum
        self.vel = [[np.zeros_like(layer.w), 
                     np.zeros_like(layer.b)] for layer in model.layers]

    def step(self):
        for i, layer in enumerate(self.model.layers):
            self.vel[i][0] = self.vel[i][0] * self.m - self.lr * layer.d_w 
            self.vel[i][1] = self.vel[i][1] * self.m - self.lr * layer.d_b 
            layer.w += self.vel[i][0]
            layer.b += self.vel[i][1]
    
    def zero_grad(self):
        self.model._clear_state()

In [ ]:
# pred = model(x)
# loss = criterion(pred, target)
# grad = d loss / d pred
# model.backward(grad)
# optim.step()

In [33]:
x = np.random.uniform(-2, 2, 20000)
y = x**2 + np.random.randn()*0.1


In [61]:
model = Model(((1, 100), (100, 1)), activation='relu')
optim = SGDMomentum(model)
for e in range(20):
    for i, (val, t) in enumerate(zip(x, y)):
        optim.zero_grad()
        pred = model.forward(np.array([[val]]))
        loss = mse_loss(t, pred)
        grad = d_mse_loss(t, pred)
        model.backward(grad)
        optim.step()
        
    print(e, model.forward([[1]]), model.forward([[2]]), model.forward([[-1]]), model.forward([[-2]]))

0 [[1.34408245]] [[2.45722998]] [[1.3377181]] [[2.56048745]]
1 [[1.29793384]] [[3.04754323]] [[1.26022792]] [[3.11194714]]
2 [[1.25546854]] [[3.23938504]] [[1.21719352]] [[3.28644816]]
3 [[1.22713088]] [[3.31119813]] [[1.19175437]] [[3.3463366]]
4 [[1.20084985]] [[3.35432575]] [[1.16689864]] [[3.38184866]]
5 [[1.17315276]] [[3.39125805]] [[1.13976081]] [[3.41485538]]
6 [[1.14430877]] [[3.42593893]] [[1.11108374]] [[3.4476508]]
7 [[1.11579012]] [[3.45862088]] [[1.08245985]] [[3.47961451]]
8 [[1.0878903]] [[3.48847209]] [[1.05477301]] [[3.50941747]]
9 [[1.06113239]] [[3.515334]] [[1.0286288]] [[3.53676979]]
10 [[1.0357463]] [[3.53946749]] [[1.00424153]] [[3.56131824]]
11 [[1.0119417]] [[3.56174452]] [[0.9819401]] [[3.58490369]]
12 [[0.99380215]] [[3.58140495]] [[0.96134751]] [[3.6047272]]
13 [[0.98127463]] [[3.59915268]] [[0.94452716]] [[3.6216558]]
14 [[0.97061183]] [[3.61554108]] [[0.93161373]] [[3.636704]]
15 [[0.96090792]] [[3.63049534]] [[0.92214808]] [[3.65016956]]
16 [[0.95161455]